In [ ]:
%reload_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd
import numpy as np
from dmyplant2 import (
    cred, MyPlant, Engine, msgFSM, FSM, bokeh_show, get_cycle_data2, get_cycle_data, v, chart, FSMPlot_Start, dbokeh_chart, FSM_splot
)
from pprint import pprint as pp
import time


import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='simple_white'
pio.renderers.default = "notebook"
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();

In [ ]:
pio.templates;

In [ ]:
fleet = mp.search_installed_fleet_by_contains_name("Forsa Hartmoor")
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
motor = fleet.iloc[3]
modes = ['???','OFF','MANUAL','AUTO']
success = [True,False]
#success = [True]
fleet.T
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
t0 = time.time(); e=Engine.from_fleet(mp,motor);t1 = time.time()
fsm = msgFSM(e, p_from=e['Commissioning Date'], p_to='2022-01-15', successtime=300); t2 = time.time()
fsm.run1(enforce=True, silent=False); t3 = time.time()
print(f"engine: {t1-t0:3.2f} sec\n   fsm: {t2-t1:3.2f} sec\n  run1: {t3-t2:3.2f} sec")

In [ ]:
rda = fsm.starts
rda['stime'] = rda.apply(lambda x: x['starttime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda['etime'] = rda.apply(lambda x: x['endtime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda[['no','success','mode','stime','etime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','targetoperation','rampdown','coolrun','runout']].round(2)

In [ ]:
from dowork import plot_now
dset = [{'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['dodgerblue','red'], 'unit':'kW'},
        {'col':['power_diff'],'ylim': [-300, 300], 'color':'black', 'unit':'kW/s'},
        {'col':['Various_Bits_CollAlarm'],'ylim': [-5, 5], 'color':'black', 'unit':'-'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_TempOil'],'ylim': [0, 100], 'color':'violet', 'unit':'°C'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 10], 'color':'midnightblue', 'unit':'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'dodgerblue', 'unit':'bar'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue','unit':'rpm'}]
startversuch = rda.iloc[8];
data = get_cycle_data(fsm, startversuch, max_length=None, min_length=None, cycletime=1, silent=True, p_data=v(mp,dset))

In [ ]:
data.head(5)

In [ ]:
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
#fig = FSM_splot(fsm, startversuch, data, dset, figsize=(16,5))
#fig = FSMPlot_Start(fsm, startversuch, data, v(mp,dset), dset, figsize=(16,10))
fig = plot_now(fsm, data, startversuch,v(mp,dset), dset, dfigsize=(20,14))
bokeh_show(fig)

In [ ]:
from dowork import plot_plotly
fig2 = plot_plotly(fsm, data, startversuch, v(mp,dset), dset, dfigsize=(18,12))
fig2.show(config={'scrollZoom': True})

In [ ]:
from dowork import plot_plotly
dvset = ['Various_Values_SpeedAct','Power_PowerAct','Power_SetPower','Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_PressOil','Hyd_TempOil']
ddset = [{'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['dodgerblue','red'], 'unit':'kW'},
        {'col':['Hyd_TempOil'],'ylim': [0, 100], 'color':'violet', 'unit':'°C'},
        {'col':['Hyd_PressOil','Hyd_PressOilDif'],'ylim': [0, 10], 'color':['indigo','darkorange'], 'unit':'bar'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue','unit':'rpm'}]
data = get_cycle_data(fsm, startversuch, max_length=None, min_length=None, cycletime=1, silent=True, p_data=dvset)
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
fig2 = plot_plotly(fsm, data, startversuch,dvset, ddset, dfigsize=(12,8))
config = dict({'scrollZoom': True})
fig2.add_vline(x=startversuch['endtime'],line_dash='solid',line_width=1,line_color='red')
fig2.add_hline(y=1500.0,yref='y2',ysizemode='scaled',line_dash='solid',line_color='blue',line_width=3)
fig2.show(config=config)
#[e for rec in ddset for e in rec['col']]

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

playout = {
    'hovermode':"x unified",
    'showlegend':True,
    'legend': { 'yanchor':'top', 'y':0.99,'xanchor':'left', 'x':0.01,'bgcolor': 'rgba(255,255,255,0.4)'},
    'title_text': f"{fsm._e} -- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}",
    'width':1000,
    'height':600,
    'hoverlabel':{
        'bgcolor': 'rgba(240,240,240,0.7)',
        'font_family': 'Courier',
        'namelength': -1,
        },
    'xaxis':{ 'domain':[0,0.85]},
    'yaxis':{
        'title':'PowerAct, SpeedAct',
        #'color':"#1f77b4",
        'color':"black",
        'anchor':"x",
        'overlaying':"y",'side':"left",'position':0.0,
        'title_standoff': 4,
        'range':(0,5000),
        },
    'yaxis2':{
        'title':"PressOil",
        'color':"#d62728",
        'anchor':"free",'overlaying':"free",'side':"right",'position':0.85,
        'title_standoff': 4,
        'range':(0,10),
        'showgrid':True,
        'showline':True
        },
    'yaxis3':{
        'title':"TempOil",
        'color':"#9467bd",
        'anchor':"free",'overlaying':"y",'side':"right",'position':0.95,
        'title_standoff': 4,
        'range':(0,100),
        },
    # 'yaxis4':{
    #     'title':"SpeedAct",
    #     'color':"#ff7f0e",
    #     'anchor':"free",
    #     'overlaying':'y','side':"right",'position':0.7,
    #     'range':(0,2000),
    #     },
    'yaxis4':{
        'title':"",
        'color':"#ffffff",
        'anchor':"free",
        'overlaying':'y','side':"right",'position':1.0,
        'title_standoff': 4,
        'range':(0,100),
        'showline':False,
        'nticks':0
        },
}

data['hover'] = 60

pdata = [
    go.Scattergl(x=data['datetime'],y=data['Power_PowerAct'], name='PowerAct',yaxis="y"), #Power_PowerAct
    go.Scattergl(x=data['datetime'],y=data['Various_Values_SpeedAct'], name="SpeedAct",yaxis="y"),
    go.Scattergl(x=data['datetime'],y=data['Hyd_PressOil'],name="PressOil",yaxis="y2"),
    go.Scattergl(x=data['datetime'],y=data['Hyd_TempOil'],name="TempOil",yaxis="y3"),
    go.Scattergl(x=data['datetime'],y=data['hover'], line_color='rgba(255,255,255,0)', name="",yaxis="y4"),
]

#customdata=np.stack((data['Power_PowerAct'], data['Various_Values_SpeedAct']), axis=-1)
subdata=data[['Power_PowerAct','Various_Values_SpeedAct','Hyd_PressOil','Hyd_TempOil']]
customdata=[tuple(x) for x in subdata.to_numpy()]
hovertemplate = ("Power:    %{customdata[0]:5.0f} [kW]<br>" +
                 "Speed:    %{customdata[1]:5.0f} [rpm]<br>" +
                 "Oilpress: %{customdata[2]:5.2f} [bar]<br>" +
                 "Oiltemp:  %{customdata[3]:5.1f} [°C]<br>" +
                 "<extra></extra>")

fig3 = go.Figure(data=pdata,layout=playout)
fig3.update_traces(customdata=customdata, hovertemplate=hovertemplate)
fig3.show()


In [ ]:
chart(data,dset, figsize=(16,10));

In [ ]:
svdetail=fsm.starts.iloc[6].to_dict()
#pp(svdetail);

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
print(fsm.results['runlogdetail'][0])
print(fsm.results['runlogdetail'][-1])

In [ ]:
print(len(fsm.results['runlogdetail']))